# How to Setup Amazon SageMaker with MLflow

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook.

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/sagemaker-mlflow|sagemaker_mlflow_setup.ipynb)

## Updates and Imports

In [1]:
#!pip install -U --quiet boto3

Imports

In [2]:
import json
import sagemaker
import boto3

print(json.__version__)
print(sagemaker.__version__)
print(boto3.__version__)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
2.0.9
2.227.0
1.34.162


Session variables

In [3]:
sess = sagemaker.Session()
bucket_name = sess.default_bucket()
role = sagemaker.get_execution_role()
region = sess.boto_region_name

iam_client = boto3.client("iam")
sts_client = boto3.client("sts")
sm_client = boto3.client("sagemaker")
account_id = sts_client.get_caller_identity()["Account"]
tracking_server_name = "my-tracking-server-No1"
mlflow_role_name = "my-mlflow-role-No1"
mlflow_policy_name = "mlflow-s3-sm-model-registry"

## MLflow Permissions

### IAM Role for the MLflow Tracking Server

To run the next cell, make sure the IAM role used while running this notebook has permission to create an IAM Role. 
The `iam:CreateRole`, `iam:CreatePolicy`, `iam:ListPolicies`, and `iam:AttachRolePolicy` action must be allowed by the notebook execution role's policy.

If you are running this notebook from SageMaker Studio, you can update your notebook execution role through the following steps: 

1. Navigate to the AWS Console and select the Domain you are using
2. Under the Domain, select the User Profile you are using. You will see the Execution Role listed there.
3. Navigate to the IAM Console, search for the Execution Role under "Roles", and update your role with a policy that allows the `iam:CreateRole`, `iam:CreatePolicy`, `iam:ListPolicies`, and `iam:AttachRolePolicy` actions. 

If you are not using a SageMaker Studio Notebook, confirm that the role you have used to configure your AWS CLI has appropriate permissions to create an IAM role and attach a policy to it. 

Here is an example of an inline policy you can add to your role - 

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "Statement1",
            "Effect": "Allow",
            "Action": [
                "iam:ListPolicies",
                "iam:CreatePolicy",
                "iam:CreateRole",
                "iam:AttachRolePolicy"
            ],
            "Resource": [
                "*"
            ]
        }
    ]
}
```

In [4]:
mlflow_trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {"Service": ["sagemaker.amazonaws.com"]},
            "Action": "sts:AssumeRole",
        }
    ],
}

# Create role for MLflow
try:
    # 기존 역할이 있는지 확인
    existing_role = iam_client.get_role(
    RoleName=mlflow_role_name)
    print(f"Role {mlflow_role_name} already exists")
    mlflow_role = existing_role["Role"]
except iam_client.exceptions.NoSuchEntityException:
    # 역할이 없는 경우에만 새로 생성
    mlflow_role = iam_client.create_role(
        RoleName=mlflow_role_name,
        AssumeRolePolicyDocument=json.dumps(mlflow_trust_policy)
    )
    print(f"Created new role: {mlflow_role_name}")

mlflow_role_arn = mlflow_role["Arn"]

Role my-mlflow-role-No1 already exists


In [5]:
# Create policy for S3 and SageMaker Model Registry
sm_s3_model_registry_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:Get*",
                "s3:Put*",
                "s3:List*",
                "sagemaker:AddTags",
                "sagemaker:CreateModelPackageGroup",
                "sagemaker:CreateModelPackage",
                "sagemaker:UpdateModelPackage",
                "sagemaker:DescribeModelPackageGroup",
            ],
            "Resource": "*",
        }
    ],
}

try:
    # 기존 정책이 있는지 확인
    existing_policy = iam_client.get_policy(
        PolicyArn=f"arn:aws:iam::{account_id}:policy/{mlflow_policy_name}"
    )
    print(f"Policy {mlflow_policy_name} already exists")
    mlflow_s3_sm_model_registry_iam_policy = existing_policy["Policy"]
except iam_client.exceptions.NoSuchEntityException:
    # 정책이 없는 경우에만 새로 생성
    mlflow_s3_sm_model_registry_iam_policy = iam_client.create_policy(
        PolicyName=mlflow_policy_name,
        PolicyDocument=json.dumps(sm_s3_model_registry_policy)
    )
    print(f"Created new policy: {mlflow_policy_name}")

#mlflow_s3_sm_model_registry_iam_policy = iam_client.create_policy(
#    PolicyName={mlflow_policy_name}, PolicyDocument=json.dumps(sm_s3_model_registry_policy)
#)

mlflow_s3_sm_model_registry_iam_policy_arn = mlflow_s3_sm_model_registry_iam_policy["Arn"]

Policy mlflow-s3-sm-model-registry already exists


In [6]:
# Attach the policy to the MLflow role
iam_client.attach_role_policy(
    RoleName=mlflow_role_name, PolicyArn=mlflow_s3_sm_model_registry_iam_policy_arn
)

{'ResponseMetadata': {'RequestId': '5a6348a1-0867-4862-be7e-8c009d5215d1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 05 Jan 2025 00:40:38 GMT',
   'x-amzn-requestid': '5a6348a1-0867-4862-be7e-8c009d5215d1',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

Note that your SageMaker execution role should have the following permissions to call Mlflow REST APIs:

```json
{
    "Version": "2012-10-17",    
    "Statement": [        
        {            
            "Effect": "Allow",            
            "Action": [
                "sagemaker-mlflow:*",
                "sagemaker:CreateMlflowTrackingServer",
                "sagemaker:UpdateMlflowTrackingServer",
                "sagemaker:DeleteMlflowTrackingServer",
                "sagemaker:StartMlflowTrackingServer",
                "sagemaker:StopMlflowTrackingServer",
                "sagemaker:CreatePresignedMlflowTrackingServerUrl"
            ],            
            "Resource": "*"        
        }        
    ]
}
```

## Create MLflow Tracking Server

In [7]:
sm_client.create_mlflow_tracking_server(
    TrackingServerName=tracking_server_name,
    ArtifactStoreUri=f"s3://{bucket_name}/{tracking_server_name}",
    TrackingServerSize="Small",
    MlflowVersion="2.13.2",
    RoleArn=mlflow_role_arn,
    AutomaticModelRegistration=False,
)

{'TrackingServerArn': 'arn:aws:sagemaker:us-east-1:211125368524:mlflow-tracking-server/my-tracking-server-No1',
 'ResponseMetadata': {'RequestId': '63ae8448-fca0-4dce-b1b0-7c01ecd9a720',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '63ae8448-fca0-4dce-b1b0-7c01ecd9a720',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '110',
   'date': 'Sun, 05 Jan 2025 00:40:39 GMT'},
  'RetryAttempts': 0}}

In [8]:
tracking_server_arn = (
    f"arn:aws:sagemaker:{region}:{account_id}:mlflow-tracking-server/{tracking_server_name}"
)

Amazon SageMaker의 MLflow 트래킹 서버의 구성 및 상세 정보를 확인

In [9]:
sm_client.describe_mlflow_tracking_server(TrackingServerName=tracking_server_name)

{'TrackingServerArn': 'arn:aws:sagemaker:us-east-1:211125368524:mlflow-tracking-server/my-tracking-server-No1',
 'TrackingServerName': 'my-tracking-server-No1',
 'ArtifactStoreUri': 's3://sagemaker-us-east-1-211125368524/my-tracking-server-No1',
 'TrackingServerSize': 'Small',
 'MlflowVersion': '2.13.2',
 'RoleArn': 'arn:aws:iam::211125368524:role/my-mlflow-role-No1',
 'TrackingServerStatus': 'Creating',
 'IsActive': 'Inactive',
 'TrackingServerUrl': 'https://t-wpgo5ybajp7h.us-east-1.experiments.sagemaker.aws',
 'WeeklyMaintenanceWindowStart': 'Sun:20:23',
 'AutomaticModelRegistration': False,
 'CreationTime': datetime.datetime(2025, 1, 5, 0, 40, 39, 728000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:211125368524:user-profile/d-zclmp6rakgrp/default-20240903T101789',
  'UserProfileName': 'default-20240903T101789',
  'DomainId': 'd-zclmp6rakgrp'},
 'LastModifiedTime': datetime.datetime(2025, 1, 5, 0, 40, 39, 728000, tzinfo=tzlocal()),
 'LastModifiedB

Install the MLflow SDK and our MLflow AWS Plugin (SageMaker Studio Space에서는 생략 가능)

In [10]:
#!pip install --quiet mlflow==2.13.2 sagemaker-mlflow==0.1.0

## MLflow tracking test

Connect to tracking server

In [11]:
import mlflow

mlflow.set_tracking_uri(tracking_server_arn)

Log a metric (mlflow tracking server 생성 완료 후 실행)

In [13]:
with mlflow.start_run():
    mlflow.log_metric("foo", 1)

2025/01/05 01:26:15 INFO mlflow.tracking._tracking_service.client: 🏃 View run melodic-quail-776 at: https://us-east-1.experiments.sagemaker.aws/#/experiments/0/runs/e1e7f8ea4c074a839bb196338ba7c700.
2025/01/05 01:26:15 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://us-east-1.experiments.sagemaker.aws/#/experiments/0.


See results in MLflow UI. You can either launch the MLflow UI from within SageMaker Studio, or generate a pre-signed URL like this:

In [14]:
sm_client.create_presigned_mlflow_tracking_server_url(TrackingServerName=tracking_server_name)

{'AuthorizedUrl': 'https://t-wpgo5ybajp7h.us-east-1.experiments.sagemaker.aws/auth?authToken=eyJhbGciOiJIUzI1NiJ9.eyJhdXRoVG9rZW5JZCI6IjI1NmY2MzQ3LTZlMjUtNDY2OC1hODg4LTlkN2MwNDVlODZmMSIsImZhc0NyZWRlbnRpYWxzIjoiQWdWNEd1RHFsbjMwRHZJS3NNZkM0bU5EMVp2a0JrNy9vM3hndDFGNEsxWnFDcDBBWHdBQkFCVmhkM010WTNKNWNIUnZMWEIxWW14cFl5MXJaWGtBUkVGNVMyTlpVMXBvVW5sSFZVRmFVSE5NVURSblMwMUVZelZQY2pJd2VtSlBUbmRPVVZGV2IwbExZV0pWV1dnMWJtczROVzlxWjJOWVVWRjFaWE5FUW1neGR6MDlBQUVBQjJGM2N5MXJiWE1BUzJGeWJqcGhkM002YTIxek9uVnpMV1ZoYzNRdE1UbzROVEUzTWpVME56TTFNelU2YTJWNUwyRTVZMkZrWVRjNUxXTTVaRFl0TkRGbU5TMDRNakl5TFRsak1qYzNaVGMyWkdRMllRQzRBUUlCQUhnaVlibDFFZzUwbUordEE5TzVldXgramlSdENidVNrMXhNZWVnN3M0cnBTd0VxU0trSnFZR3BPdTIxWGJSTkNkUXpBQUFBZmpCOEJna3Foa2lHOXcwQkJ3YWdiekJ0QWdFQU1HZ0dDU3FHU0liM0RRRUhBVEFlQmdsZ2hrZ0JaUU1FQVM0d0VRUU1PZUxnQkxnWkdCeDJDQUhaQWdFUWdEdENxZURxc0FEVFB6NUUxTWoyZGREV1hjNG1JcWhzSVFiZXlUUCtKQmM5WjhrV3pQQkg0bms1a04rYm1lM09tMU1rMk5DL2Zndk1qdVZyQ0FJQUFCQUFVZGlZUExKN250ZXV4YUdWUFVQR2txSU9jR0xPdTB2MXRJWll0cElkWVhrTW

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.


![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/sagemaker-mlflow|sagemaker_mlflow_setup.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/sagemaker-mlflow|sagemaker_mlflow_setup.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/sagemaker-mlflow|sagemaker_mlflow_setup.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/sagemaker-mlflow|sagemaker_mlflow_setup.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/sagemaker-mlflow|sagemaker_mlflow_setup.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/sagemaker-mlflow|sagemaker_mlflow_setup.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/sagemaker-mlflow|sagemaker_mlflow_setup.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/sagemaker-mlflow|sagemaker_mlflow_setup.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/sagemaker-mlflow|sagemaker_mlflow_setup.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/sagemaker-mlflow|sagemaker_mlflow_setup.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/sagemaker-mlflow|sagemaker_mlflow_setup.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/sagemaker-mlflow|sagemaker_mlflow_setup.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/sagemaker-mlflow|sagemaker_mlflow_setup.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/sagemaker-mlflow|sagemaker_mlflow_setup.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/sagemaker-mlflow|sagemaker_mlflow_setup.ipynb)